# LAMMPS 

In [1]:
from lammps import lammps
from comafunctions_Lammps import animate, preview_structure
import numpy as np
import time
import os 

In [2]:
lmp = lammps(cmdargs=['-sc', 'log']) 

## Initial Structure

In [3]:
preview_structure('structures/Ti_Initial.txt')

Number of structured atoms: 972


JupyterViewportWidget(camera_params={'fov': 22.62764276573904, 'perspective': False, 'matrix': [[0.70710678118…

## LAMMPS Input Script
### https://docs.lammps.org/Commands_all.html

In [4]:
t_start = 1 #Kelvin
t_end = 2000 #Kelvin
timestep = 0.001
thermo = 200
steps = 40000
element = 'Ti'

### Deformation Al

In [ ]:
cmd = f'''
units metal
dimension 3
boundary p p p
atom_style atomic
read_data structures/crystalline_fcc.inp
pair_style eam/alloy #adp
pair_coeff * * potentials/Al03.eam.alloy Al#Si_Au_Al.adp.txt Al Al Al

compute csym all centro/atom fcc
compute peratom all pe/atom

reset_timestep 0
timestep 0.001
velocity all create 300 12345 mom yes rot no
fix 1 all npt temp 300 300 1 iso 0 0 1 drag 1

thermo 1000

run 1000

unfix 1

reset_timestep 0

fix      1 all npt temp 300 300 1 y 0 0 1 z 0 0 1 drag 1
variable srate equal 1.0e10
variable srate1 equal "v_srate / 1.0e12"
fix      2 all deform 1 x erate 0.001 units box remap x #$[srate1]

dump     1 all cfg 20000 dump.tensile_*.cfg mass type xs ys zs c_csym c_peratom fx fy fz
#dump id all atom 50 dump2.lammpsstrj

thermo 1000

run 20000

'''

### Periodic Boundary Conditions Al Heating

In [5]:
cmd = f'''
units metal
dimension 3
boundary p p p
atom_style atomic
read_data structures/crystalline_fcc.inp
pair_style eam/alloy #adp
pair_coeff * * potentials/Al03.eam.alloy Al#Si_Au_Al.adp.txt Al Al Al

timestep {timestep}

thermo {thermo}

thermo_style custom step temp vol press ke pe density 

variable s equal step
variable t equal temp
variable v equal vol
variable p equal press
variable k equal ke
variable P equal pe
variable d equal density
variable e equal etotal


fix thermo all print 200 '$s $t $v $p $k $P $d $e' file output/thermo_output_{element}.dat screen no

velocity all create 1 589500 mom yes rot yes dist gaussian

dump 1 all custom 200 output/dump_{element}.out id type xs ys zs

fix 2 all nvt temp {t_start} {t_end} 0.1

run {steps}

'''

### Non Periodic Boundary Conditions Al Heating

In [6]:
cmd = f'''

dimension 3
units metal
atom_style atomic
boundary p p p

read_data structures/crystalline_fcc.inp

pair_style eam/alloy #adp
pair_coeff * * potentials/Al03.eam.alloy Al#Si_Au_Al.adp.txt Al Al Al

neighbor 3.0 bin

write_data original

change_box all x delta 0 3 
change_box all y delta 0 3 
change_box all z delta 0 3 boundary s s s

timestep {timestep}

thermo {thermo}

thermo_style custom step temp vol press ke pe density 

variable s equal step
variable t equal temp
variable v equal vol
variable p equal press
variable k equal ke
variable P equal pe
variable d equal density
variable e equal etotal


fix thermo all print 200 '$s $t $v $p $k $P $d $e' file output/thermo_output_{element}.dat screen no

velocity all create 1 589500 mom yes rot yes dist gaussian

dump 1 all custom 200 output/dump_{element}.out id type xs ys zs

fix 2 all nvt temp {t_start} {t_end} 0.1

run {steps}

#unfix 2

#fix 2 all nvt temp {t_end} {t_start} 0.1

#run 15000
       
'''

### Solid Phase Transfromation Ti Heating

In [5]:
cmd = f'''
dimension 3
units metal
atom_style atomic
boundary p p p
box tilt large

read_data structures/Ti_Initial.txt

pair_style meam
pair_coeff * * potentials/libraryNiTi.meam Ni Ti potentials/NiTi.meam Ti

#pair_style eam/alloy
#pair_coeff * * Mendelev-M-I-Underwood-T-L-Ackland-G-J--Ti-1 Ti#Farkas_Nb-Ti-Al_1996.eam.alloy Ti

write_data original

timestep 0.001

fix ensemble all npt temp {t_start} {t_end} 0.1 x 10.0 10.0 1.0 y 10.0 10.0 1.0 z 10.0 10.0 1.0 xy 0.0 0.0 1.0 xz 0.0 0.0 1.0 yz 0.0 0.0 1.0

velocity all create 1 589500 dist gaussian

dump 1 all custom 200 output/dump_{element}.out id type xsu ysu zsu fx fy fz vx vy vz
dump_modify 1 sort id format line "%d %d %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g %20.15g"
thermo_style custom step temp pe etotal vol
# thermo_style custom step temp pe etotal pxx pxy pxz pyy pyz pzz vol
thermo_modify format float %20.15g
thermo 200

variable t equal step
variable m equal temp
fix thermo all print 200 '$t $m' file output/thermo_output_{element}.dat screen no

run 10000 


fix ensemble all npt temp {t_end} {t_start} 0.1 x 10.0 10.0 1.0 y 10.0 10.0 1.0 z 10.0 10.0 1.0 xy 0.0 0.0 1.0 xz 0.0 0.0 1.0 yz 0.0 0.0 1.0

run 20000


'''

#### Option 2

In [5]:
cmd = f'''
dimension 3
units metal
atom_style atomic
boundary p p p
box tilt large

read_data structures/crystalline_bcc.inp#Ti_Initial.txt

pair_style eam/alloy
pair_coeff * * potentials/Farkas_Nb-Ti-Al_1996.eam.alloy Ti

timestep {timestep}

thermo {thermo}

thermo_style custom step temp vol press ke pe density 

variable s equal step
variable t equal temp
variable v equal vol
variable p equal press
variable k equal ke
variable P equal pe
variable d equal density
variable e equal etotal


fix thermo all print 200 '$s $t $v $p $k $P $d $e' file output/thermo_output_{element}.dat screen no

velocity all create 1 589500 mom yes rot yes dist gaussian

dump 1 all custom 200 output/dump_{element}.out id type xs ys zs

fix 2 all nvt temp {t_end} {t_start} 0.1

run 20000 #{steps}


#unfix 2

#fix 2 all nvt temp {t_end} {t_start} 0.1

#run 30000

'''

## Run and Animate

In [6]:
start_time = time.time()

#Run the simulation
lmp.commands_string(cmd)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.6f} seconds")

Elapsed time: 902.007443 seconds


In [ ]:
name_dump = 'Phasetransformation_fast'
name_thermo = 'Phasetransformation_fast'
os.rename(f'output/dump_{element}.out',f'output/dump_{element}_{name_dump}.out')
os.rename(f'output/thermo_output_{element}.dat',f'output/thermo_output_{element}_{name_thermo}.dat')

In [7]:
animate(f'dump_{element}_{name_dump}.out',f'thermo_output_{element}_{name_thermo}.dat')

Number of FCC atoms: 972


AppLayout(children=(HTML(value='<h2>Animation of Results</h2>', layout=Layout(grid_area='header', height='5px'…